# Iteration 1: modelling unplanned arrivals

## Plan English Summary

We increased the scope of the model by including a treatment process for unplanned (emergency) arrivals to the model.  Like arrrivals each type of patient has their own treatment distribution for sampling.  The treatment process also introduces a shared resource into the model - critical care beds - and a First come first served queue.

The input command is explicit that the code should be *modified* and that the full code listing should be produced.




## Notebook imports

In [1]:
from IPython.display import Markdown, display

## Show query 

In [2]:
display(Markdown("inputs/iteration_1.md"))

### Main command

Modify the model to add in an unplanned admissions treatment process.

Show the full code listing.  Do not hide any functions or classes.

### General model logic

The model includes a resource: critical care beds.

After patients have arrived they follow the unplanned admissions process. In the unplanned admissions process patients request and wait for a critical care bed resource to become available in a FIFO queue, and undergo treatment.  Following treatment the patient is recorded as "discharged", but the bed is not released until intensive cleaning has taken place.  

The model should print out useful information after each event.

### Simulation inputs

the length of stay (measured in hours) for treatment of patients in the CCU follow source specific distributions

Accident and Emergency: Lognormal: mean 128.79 and standard deviation 267.51
the Wards = Lognormal: mean 177.89 and standard deviation 276.54
Emergency surgery = Lognormal: mean 140.15 and standard deviation 218.02
other hospitals = Lognormal: mean 212.86 and standard deviation 457.67
the X-Ray department = Lognormal: mean 87.53 and standard deviation 108.67

The mean and standard deviation of the Lognormal distributions must be converted to the mean and standard deviation of the underlying normal distribution.

There are 24 critical care beds in the model.  These are shared across all types of patient.

Intensive cleaning takes a fixed amount of time: 5 hours.




## Date and time of input to LLM

> 1233 27th November 2023 

## Observations on the LLM Output

* 

## Sources reported by the LLM

*

## LLM Output

In [4]:
import simpy
import numpy as np

# Define the critical care unit (CCU) model with unplanned admissions treatment process
class CCUModel:
    def __init__(self, env, run_length=12*30*24):  # Default run length is 12 months
        self.env = env
        self.run_length = run_length
        self.sources = ['Accident and Emergency', 'the Wards', 'Emergency surgery', 'other hospitals', 'the X-Ray department']
        self.interarrival_distributions = [22.72, 26.0, 37.0, 47.2, 575.0]
        self.stay_distributions = [(128.79, 267.51), (177.89, 276.54), (140.15, 218.02), (212.86, 457.67), (87.53, 108.67)]
        self.patient_count = 0
        self.critical_care_beds = simpy.Resource(env, capacity=24)

    # Generator function for patient arrival from Accident and Emergency
    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(22.72))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[0]))

    # Generator function for patient arrival from the Wards
    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(26.0))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[1]))

    # Generator function for patient arrival from Emergency surgery
    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(37.0))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[2]))

    # Generator function for patient arrival from other hospitals
    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(47.2))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[3]))

    # Generator function for patient arrival from the X-Ray department
    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(575.0))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[4]))

    # Unplanned admissions treatment process
    def unplanned_admission(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2),
                                                 np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
            yield self.env.timeout(treatment_time)
            print(f"Patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(5)  # Intensive cleaning time
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    # Run the simulation
    def run(self):
        env.process(self.patient_arrival_AE())
        env.process(self.patient_arrival_wards())
        env.process(self.patient_arrival_surgery())
        env.process(self.patient_arrival_other_hospitals())
        env.process(self.patient_arrival_X_ray())
        env.run(until=self.run_length)

# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the CCU model and run the simulation
ccu_model = CCUModel(env)
ccu_model.run()

Patient 1 arrived from Accident and Emergency at time 5.838543174963645
Patient 1 admitted to critical care bed at time 5.838543174963645
Patient 2 arrived from Emergency surgery at time 8.350409617431035
Patient 2 admitted to critical care bed at time 8.350409617431035
Patient 3 arrived from the Wards at time 9.16825689100151
Patient 3 admitted to critical care bed at time 9.16825689100151
Patient 4 arrived from the Wards at time 34.51933623686176
Patient 4 admitted to critical care bed at time 34.51933623686176
Patient 4 discharged from critical care bed at time 41.65498332183833
Critical care bed is available for next patient at time 46.65498332183833
Patient 4 discharged from critical care bed at time 49.0532350431315
Patient 4 discharged from critical care bed at time 50.49637909924342
Critical care bed is available for next patient at time 54.0532350431315
Critical care bed is available for next patient at time 55.49637909924342
Patient 5 arrived from other hospitals at time 80.9